In [19]:
import ROOT
from math import log, sqrt

# silence most of the roofit message (bug in ROOT saturate jupyter stream)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.NumIntegration)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Fitting)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Minimization)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.InputArguments)
ROOT.RooMsgService.instance().getStream(1).removeTopic(ROOT.RooFit.Eval)
ROOT.RooMsgService.instance().setGlobalKillBelow(ROOT.RooFit.ERROR)

ROOT.RooStats.AsymptoticCalculator.SetPrintLevel(-1)

ROOT.enableJSVis()

In [20]:
f = ROOT.TFile.Open('workspace.root')
ws = f.Get("ws")
ws.Print()


RooWorkspace(ws) ws contents

variables
---------
(bkg_slope,global_efficiency,global_luminosity,global_resolution,mH,mass,nominal_efficiency,nominal_luminosity,nominal_resolution,sigma_efficiency,sigma_luminosity,sigma_resolution,theta_efficiency,theta_luminosity,theta_resolution,xsection_bkg,xsection_x_br)

p.d.f.s
-------
RooExponential::background_shape[ x=mass c=bkg_slope ] = 0.0725556
RooGaussian::constrain_efficiency[ x=global_efficiency mean=theta_efficiency sigma=1 ] = 1
RooGaussian::constrain_luminosity[ x=global_luminosity mean=theta_luminosity sigma=1 ] = 1
RooGaussian::constrain_resolution[ x=global_resolution mean=theta_resolution sigma=1 ] = 0.990508
RooProdPdf::constraints[ constrain_luminosity * constrain_resolution * constrain_efficiency ] = 0.990508
RooProdPdf::model[ phys_pdf * constraints ] = 0.0715093
RooAddPdf::phys_pdf[ nsignal * signal_shape + nbkg * background_shape ] = 0.0721946
RooGaussian::signal_shape[ x=mass mean=peak sigma=resolution ] = 3.99195e-06

fu

## Compute intervals on POI and plot profile likelihood ratio
Repeat with / without systematics

In [3]:
# when doing interference you need only model config and data
model_config = ws.obj('model_config')
data_asimov = ws.data('data_asimov')

canvas = ROOT.TCanvas()
poi = model_config.GetParametersOfInterest().first()  # the cross section
# do an initial fit
model_config.GetPdf().fitTo(data_asimov, ROOT.RooFit.PrintLevel(-1))

# create a profile likelihood calculator (it just compute the profile
# likelihood, can perform a scan and find a confidence interval
pl = ROOT.RooStats.ProfileLikelihoodCalculator(data_asimov, model_config)
pl.SetConfidenceLevel(0.683)  # usual 1 sigma gaussian
interval = pl.GetInterval()   # the real work
lowerLimit = interval.LowerLimit(poi)
upperLimit = interval.UpperLimit(poi)
print "[lower, upper] limit: [%.3f, %.3f]" % (lowerLimit, upperLimit)
plot = ROOT.RooStats.LikelihoodIntervalPlot(interval)
plot.Draw("tf1")

model_config.GetPdf().fitTo(data_asimov, ROOT.RooFit.PrintLevel(-1))
# set the pull to be constant, in this way there are no systematic in the model
# the error we get is the error as there were no systematic at all
# we assume it to be the statistical error
ROOT.RooStats.SetAllConstant(ws.set('pulls'))
pl_nosys = ROOT.RooStats.ProfileLikelihoodCalculator(data_asimov, model_config)
pl_nosys.SetConfidenceLevel(0.683)  # usual 1 sigma gaussian
interval_nosys = pl_nosys.GetInterval()
lowerLimit_nosys = interval_nosys.LowerLimit(poi)
upperLimit_nosys = interval_nosys.UpperLimit(poi)
print "[lower, upper] limit: [%.3f, %.3f]" % (lowerLimit_nosys, upperLimit_nosys)
plot_nosys = ROOT.RooStats.LikelihoodIntervalPlot(interval_nosys)
plot_nosys.Draw("tf1 same")
ROOT.RooStats.SetAllConstant(ws.set('pulls'), False)
canvas.Draw()

[lower, upper] limit: [0.082, 0.118]
[lower, upper] limit: [0.083, 0.117]


ProfileLikelihoodCalculator is not very used since it is not very flexible: difficult to tweak the profile likelihood scan plot, not possible to parallelize the computation of the likelihood for each point of the scan... Usually people do a manual scan, usually splitting the computation on different machines.

Write a loop to do the scan using the profile likelihood object

In [4]:
model_config = ws.obj('model_config')
data_asimov = ws.data('data_asimov')

nll = model_config.GetPdf().createNLL(data_asimov)
nll.createProfile(model_config.GetParametersOfInterest())
poi = model_config.GetParametersOfInterest().first()

# write the loop here changing every time the value of the poi
poi.setVal(90)
print nll.getVal()  # this is the value of the profile likelihood for a particular value of the poi

# normalize the result to the minimum, since we want the profile likelihood *ratio*

# plot it

# repeat with no systematics (fixing NP to best values, to get the best value do a fit first)

-651229.423704


## Discovery
Try to exclude the background only model

In [5]:
ROOT.gROOT.ProcessLine('.L StandardHypoTestDemo.C')

0L

### Running the StandardHypoTestDemo macro
This is a standard macro (included in ROOT) to compute p-values for discovery (and also exlusions). In compute the observed values and also the expected values (internally it generate an Asimov dataset). Since we are using as input an Asimov dataset observed and expected are the same.

In [6]:
ROOT.StandardHypoTestDemo("workspace.root",  # workspace filename
                          "ws",              # workspace name
                          "sb_model_config", # signal+background model
                          "",                # not needed: set poi to 0
                          "data_asimov",     # data name
                          2,                 # use asymtotic calculator               
                          3)                 # use profile Likelihood one sided (i.e. = 0 if mu_hat < 0)


Results HypoTestAsymptotic_result: 
 - Null p-value = 6.46663e-10
 - Significance = 6.06822
 - CL_b: 6.46663e-10
 - CL_s+b: 0.503068
 - CL_s: 7.77944e+08
Asymptotic results 
 Expected p -value and significance at -2 sigma = 2.2917e-05 significance 4.07591 sigma 
 Expected p -value and significance at -1 sigma = 1.92819e-07 significance 5.07591 sigma 
 Expected p -value and significance at 0 sigma = 6.16418e-10 significance 6.07591 sigma 
 Expected p -value and significance at 1 sigma = 7.42336e-13 significance 7.07591 sigma 
 Expected p -value and significance at 2 sigma = 3.34866e-16 significance 8.07591 sigma 


Info in <StandardHypoTestInvDemo>: The background model  does not exist
Info in <StandardHypoTestInvDemo>: Copy it from ModelConfig sb_model_config and set POI to zero


## The same using Roostats components (Asymptotic)

In [7]:
data_asimov = ws.data('data_asimov')

# the main object: as usual it needs the data and the model config
hypoCalc= ROOT.RooStats.AsymptoticCalculator(data_asimov,
                                             ws.obj('sb_model_config'),
                                             ws.obj('b_model_config'))

# with the asymptotic calculator it is assumed we are using some sort of test statistics
# based on the profile likelihood ratio. In this case we are using the version where
# if the fitted signal is negative the value of the test statistic is 0 (OneSideDiscovery)
# This means that we are never excluding the background hypothesis when the fitted signal
# is negative
hypoCalc.SetOneSidedDiscovery(True)    
htr = hypoCalc.GetHypoTest()         # the result of the interference
htr.SetPValueIsRightTail(True)       # the signal hypothesis is for high value (right)
htr.SetBackgroundAsAlt(False)
z = htr.Significance()

print "significance is %f sigma" % z

significance is 6.068224 sigma


## The same just using Asymptotic formulas

In [8]:
ws.loadSnapshot('data_fit')
# s + b fit
ws.var('xsection_x_br').setConstant(False)
r_sb = ws.pdf('model').fitTo(data_asimov, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-1))
# b-only fit
ws.var('xsection_x_br').setVal(0)
ws.var('xsection_x_br').setConstant(True)
r_b = ws.pdf('model').fitTo(data_asimov, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-1))
ws.var('xsection_x_br').setConstant(False)
print sqrt(2 * (r_b.minNll() - r_sb.minNll()))

6.06822372803


In [9]:
# (unfortunately) the workspace has a status.
# We have done some fits so its status (the values of the variables) has changed
# Reload the original status, the one after the first fit on data
ws.loadSnapshot('data_fit')
model_config = ws.obj('model_config')
original_lumi = ws.obj('nominal_luminosity').getVal()

gr = ROOT.TGraph()

for ilumi, lumi in enumerate([0.1E3, 1E3, 2E3, 5E3, 10E3, 20E3, 30E3, 50E3]):
    ws.loadSnapshot('data_fit')
    ws.var('nominal_luminosity').setVal(lumi)
    data_asimov_lumi = ws.pdf('model').generateBinned(model_config.GetObservables(),
                                                      ROOT.RooFit.ExpectedData())
    result_sb = ws.pdf('model').fitTo(data_asimov_lumi, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-1))
    ws.var('xsection_x_br').setVal(0)
    ws.var('xsection_x_br').setConstant(True)
    result_b = ws.pdf('model').fitTo(data_asimov_lumi, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-1))
    ws.var('xsection_x_br').setConstant(False)

    
    z = sqrt(2 * (result_b.minNll() - result_sb.minNll()))
    print "lumi: %.2e, #events=%d, z=%.2f" % (lumi, data_asimov_lumi.sumEntries(), z)
    
    gr.SetPoint(ilumi, lumi / 1E3, z)
    
canvas = ROOT.TCanvas()
gr.SetMarkerStyle(20)
gr.GetXaxis().SetTitle('luminosity [fb^{-1}]')
gr.GetYaxis().SetTitle('expected significance')
gr.Draw("APL")
canvas.Draw()
ws.obj('nominal_luminosity').setVal(original_lumi)

lumi: 1.00e+02, #events=1004, z=0.61
lumi: 1.00e+03, #events=10049, z=1.92
lumi: 2.00e+03, #events=20099, z=2.71
lumi: 5.00e+03, #events=50248, z=4.29
lumi: 1.00e+04, #events=100496, z=6.07
lumi: 2.00e+04, #events=200993, z=8.58
lumi: 3.00e+04, #events=301489, z=10.51
lumi: 5.00e+04, #events=502482, z=13.57


In [10]:
# this will take ~30 seconds
data_asimov = ws.data('data_asimov')

# the main object: as usual it needs the data and the model config
hypoCalc = ROOT.RooStats.FrequentistCalculator(data_asimov,
                                               ws.obj('sb_model_config'),
                                               ws.obj('b_model_config'))
hypoCalc.SetToys(1000, 200)

profll = ROOT.RooStats.ProfileLikelihoodTestStat(ws.obj('b_model_config').GetPdf())
profll.SetOneSidedDiscovery(1)

sampler = hypoCalc.GetTestStatSampler()
sampler.SetTestStatistic(profll)
sampler.SetGenerateBinned(True)

htr = hypoCalc.GetHypoTest()         # the result of the interference
htr.SetPValueIsRightTail(True)       # the signal hypothesis is for high value (right)
htr.SetBackgroundAsAlt(False)
z = htr.Significance()

print "significance is %f sigma" % z

plot = ROOT.RooStats.HypoTestPlot(htr, 100)
canvas = ROOT.TCanvas()
plot.Draw()
canvas.SetLogy()
canvas.Draw()

significance is inf sigma


Explain why Frequentist calculator is returning infinite significance

Explain why the peak at 0 for the distribution under the b-only hypothesis

In [29]:
data_binned = ws.data('data_binned')
data_asimov = ws.data('data_asimov')

import numpy as np

ws.loadSnapshot('data_fit')
model_config = ws.obj('model_config')
original_mH = ws.obj('mH').getVal()

gr_pvalue = ROOT.TGraph()
gr_significance = ROOT.TGraph()

ws.pdf('model').fitTo(data_asimov, ROOT.RooFit.PrintLevel(-1))

for imH, mH_value in enumerate(np.linspace(100, 140, 100)):
    ws.loadSnapshot('data_fit')
    ws.var('mH').setVal(mH_value)
    
    ws.pdf('model').fitTo(data_asimov, ROOT.RooFit.PrintLevel(-1))
    
    hypoCalc= ROOT.RooStats.AsymptoticCalculator(data_binned,
                                                 ws.obj('sb_model_config'),
                                                 ws.obj('b_model_config'))
    hypoCalc.SetPrintLevel(-1)
    
    hypoCalc.SetOneSidedDiscovery(True)    
    htr = hypoCalc.GetHypoTest()         # the result of the interference
    htr.SetPValueIsRightTail(True)       # the signal hypothesis is for high value (right)
    htr.SetBackgroundAsAlt(False)

    gr_pvalue.SetPoint(imH, mH_value, htr.NullPValue())
    gr_significance.SetPoint(imH, mH_value, htr.Significance())
    
    #print mH_value, htr.Significance()
    
canvas = ROOT.TCanvas()
canvas.Divide(2, 1)
canvas.cd(1)
gr_pvalue.Draw("APL")
canvas.cd(1).SetLogy()
canvas.cd(2)
gr_significance.Draw("APL")
canvas.Draw()

In [17]:
# (unfortunately) the workspace has a status.
# We have done some fits so its status (the values of the variables) has changed
# Reload the original status, the one after the first fit on data
ws.loadSnapshot('data_fit')
model_config = ws.obj('model_config')
original_mH = ws.obj('mH').getVal()

gr = ROOT.TGraph()

for imH, mH_value in enumerate(np.linspace(100, 140, 100)):
    ws.loadSnapshot('data_fit')
    ws.var('mH').setVal(mH_value)
    
    # unconditional fit (denominator)
    result_sb = ws.pdf('model').fitTo(data_binned, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-1))
    poi_hat = ws.var('xsection_x_br').getVal()
    
    ws.loadSnapshot('data_fit')
    ws.var('mH').setVal(mH_value)
    ws.var('xsection_x_br').setVal(0)
    ws.var('xsection_x_br').setConstant(True)
    result_b = ws.pdf('model').fitTo(data_binned, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-1))
    ws.var('xsection_x_br').setConstant(False)

    if (result_b.minNll() - result_sb.minNll()) < 0 or poi_hat < 0:
        z = 0
    else:
        z = sqrt(2 * (result_b.minNll() - result_sb.minNll()))
    # print "mH: %.2e, #events=%d, z=%.2f" % (mH_value, data_asimov_lumi.sumEntries(), z)
    
    gr.SetPoint(imH, mH_value, z)
    
canvas = ROOT.TCanvas()
gr.GetYaxis().SetTitle('expected significance')
gr.Draw("APL")
canvas.Draw()
ws.obj('mH').setVal(original_mH)

AttributeError: 'RooFitResult' object has no attribute 'get'